In [ ]:
!pip install peft
!pip install accelerate
!pip install bitsandBytes
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.3 MB/s eta 0:00:00


In [ ]:
  !pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=0c3e11eed4be25de5da4b39c1cd44a23090f1eb963f765bc441fe61d0c695849
  Stored in directory: /root/.cache/pip/wheels/92/a8/b7/d8a067c31a74de9ca252bbe53dea5f896faabd25d55f541037
Successfully built GPUtil


In [ ]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():

    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

# Change if needed to accomodate memory requirements!
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Set to the GPU ID (0 for T4)

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,LlamaTokenizer
from peft import PeftModel, PeftConfig
from huggingface_hub import notebook_login
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model


if 'COLAB_GPU' in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()


In [ ]:
if 'COLAB_GPU' in os.environ:
  !huggingface-cli login
else:
   notebook_login()



⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [ ]:
# Update both transformers and bitsandbytes to latest versions
'''
!pip install --upgrade transformers bitsandbytes accelerate

# If that doesn't work, try force reinstalling
!pip uninstall bitsandbytes transformers -y
!pip install transformers bitsandbytes accelerate --no-cache-dir
'''

Found existing installation: bitsandbytes 0.47.0
Uninstalling bitsandbytes-0.47.0:
  Successfully uninstalled bitsandbytes-0.47.0
Found existing installation: transformers 4.56.1
Uninstalling transformers-4.56.1:
  Successfully uninstalled transformers-4.56.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 285.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 330.4 MB/s eta 0:00:00


In [ ]:
# Diagnostic code to check bitsandbytes installation
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Test bitsandbytes import
try:
    import bitsandbytes as bnb
    print(f"✅ Bitsandbytes imported successfully: {bnb.__version__}")
except ImportError as e:
    print(f"❌ Error importing bitsandbytes: {e}")

# Test transformers import
try:
    from transformers import BitsAndBytesConfig
    print("✅ BitsAndBytesConfig imported successfully")
except ImportError as e:
    print(f"Error importing BitsAndBytesConfig: {e}")

# Check if bitsandbytes functions are available
try:
    from transformers.utils import is_bitsandbytes_available
    print(f"✅ Bitsandbytes available check: {is_bitsandbytes_available()}")
except Exception as e:
    print(f"Error checking bitsandbytes availability: {e}")

# Test creating BitsAndBytesConfig
try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    print("BitsAndBytesConfig created successfully")
except Exception as e:
    print(f"Error creating BitsAndBytesConfig: {e}")

print("\nIf all checks pass, try loading the model now...")

PyTorch version: 2.8.0+cu126
CUDA available: True
✅ Bitsandbytes imported successfully: 0.47.0
✅ BitsAndBytesConfig imported successfully
✅ Bitsandbytes available check: True
✅ BitsAndBytesConfig created successfully

If all checks pass, try loading the model now...


In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                             quantization_config=bnb_config)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
!git clone https://github.com/poloclub/Fine-tuning-LLMs.git

Cloning into 'Fine-tuning-LLMs'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 14), reused 29 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 9.34 MiB | 42.88 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
from datasets import  load_dataset
train_dataset = load_dataset("text", data_files={"train":
                ["/content/Fine-tuning-LLMs/data/hawaii_wf_1.txt", "/content/Fine-tuning-LLMs/data/hawaii_wf_2.txt",
                 "/content/Fine-tuning-LLMs/data/hawaii_wf_3.txt","/content/Fine-tuning-LLMs/data/hawaii_wf_4.txt",
                 "/content/Fine-tuning-LLMs/data/hawaii_wf_5.txt","/content/Fine-tuning-LLMs/data/hawaii_wf_6.txt",
                 "/content/Fine-tuning-LLMs/data/hawaii_wf_7.txt","/content/Fine-tuning-LLMs/data/hawaii_wf_8.txt",
                "/content/Fine-tuning-LLMs/data/hawaii_wf_9.txt","/content/Fine-tuning-LLMs/data/hawaii_wf_10.txt","/content/Fine-tuning-LLMs/data/hawaii_wf_11.txt"]}, split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dataset['text'][0]

'POLICE DEPARTMENT'

In [ ]:
tokenizer=LlamaTokenizer.from_pretrained(base_model_id,use_fast=False,trust_remote_code=True,add_eos_token=True)
#specific to llaama models
# padding cause laama models wont work on uneven datasets
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
#preprocess
tokenized_train_dataset=[]
for phrase in train_dataset:
  tokenized_train_dataset.append(tokenizer(phrase["text"]))

In [ ]:
tokenized_train_dataset[1]

{'input_ids': [1, 21122, 29979, 8079, 14861, 3120, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:

model.gradient_checkpointing_enable()

# compressing the base model into a smaller, more efficient model
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(

    # Lower rank leads to smaller matrices with fewer trainable params
    r=8,


    lora_alpha=64,

    #modules to apply the LoRA update matrices
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "down_proj",
        "up_proj",
        "o_proj"
    ],


    bias="none",

    # regulates model regularization; increasing may lead to underfitting
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
#Now ready to train Llama 2 on our data

In [ ]:
### TWO IMPORTANT TRAINING PARAMETERS TO CONSIDER CHANGING

train_epochs_val = 3
'''
train_epochs_val is the times the model will iterate over the entire training
dataset. Increasing the value may allow the model to learn more from the data,
but be cautious of overfitting.
'''

learning_rate_val = 1e-4
'''
A higher learning_rate_val can lead to faster convergence, but it might
overshoot the optimal solution. Conversely, a lower value may result
in slower training but better fine-tuning.
'''


trainer = transformers.Trainer(
    model=model,                             # llama-2-7b-chat model
    train_dataset=tokenized_train_dataset,   # training data that's tokenized
    args=transformers.TrainingArguments(
        output_dir="./finetunedModel",       # directory where checkpoints are saved
        per_device_train_batch_size=2,       # number of samples processed in one forward/backward pass per GPU
        gradient_accumulation_steps=2,       # [default = 1] number of updates steps to accumulate the gradients for
        num_train_epochs=train_epochs_val,   # [IMPORTANT] number of times of complete pass through the entire training dataset
        learning_rate=learning_rate_val,     # [IMPORTANT] smaller LR for better finetuning
        bf16=False,                          # train parameters with this precision
        optim="paged_adamw_8bit",            # use paging to improve memory management of default adamw optimizer
        logging_dir="./logs",                # directory to save training log outputs
        save_strategy="epoch",               # [default = "steps"] store after every iteration of a datapoint
        save_steps=50,                       # save checkpoint after number of iterations
        logging_steps = 10                   # specify frequency of printing training loss data
    ),

    # use to form a batch from a list of elements of train_dataset
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# if use_cache is True, past key values are used to speed up decoding

model.config.use_cache = False

# train the model based on the above config
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: akshaykumargarur (akshaykumargarur-bnm-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,4.018900
20,3.332800
30,2.882100
40,2.923100
50,2.846800
60,2.903900
70,2.896000
80,2.413900
90,2.694400
100,2.457400


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1455, training_loss=1.8560450016428105, metrics={'train_runtime': 6984.5883, 'train_samples_per_second': 0.833, 'train_steps_per_second': 0.208, 'total_flos': 6388999979163648.0, 'train_loss': 1.8560450016428105, 'epoch': 3.0})

In [ ]:
#Loading of the above fine tuned model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False,
                                           trust_remote_code=True,
                                           add_eos_token=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  #same as before
    quantization_config=nf4Config,  #same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False,
                                           trust_remote_code=True)

# Change model checkpoint that has least training loss in the code below

modelFinetuned = PeftModel.from_pretrained(base_model,"finetunedModel/checkpoint-1455")

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:


question = "Just answer this question: Tell me about the role of Maui Emergency Management Agency (MEMA) in the 2023 wildfires??"

# Format the question
eval_prompt = f"{question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Just answer this question: Tell me about the role of Maui Emergency Management Agency (MEMA) in the 2023 wildfires??

Answer: MEMA played a crucial role in coordinating the response to the 2023 wildfires. MEMA is responsible for coordinating the emergency response to natural and man-made disasters in Maui County. In response to the 2023 wildfires, MEMA worked closely with other emergency response agencies, including the Maui Fire Department, the Maui Police Department, the Hawaii National Guard, and the American Red Cross to coordinate evacuations, provide life-saving information to the public, and facilitate the flow of critical information between different agencies. MEMA also worked with media outlets to keep the public informed about the status of the fires and any evacuations that were taking place. In addition, MEMA worked to provide support to the emergency responders and the community during this time. This included providing information and resources to help the public cope wi

In [ ]:
#Interactive

In [ ]:



user_question = input("Enter your question: ")


eval_prompt = f"Just answer this question concisely: {user_question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Enter your question: Can you tell me more info about MEMA I dont know who is mema
Just answer this question concisely: Can you tell me more info about MEMA I dont know who is mema

From MEMA’s Facebook page: “MEMA is the Maui Emergency Management Agency. Our mission is to coordinate and integrate all emergency response and support activities for the safety of life and property of the people of Maui County. MEMA is responsible for managing the emergency operations center and all emergency response activities during a disaster. MEMA is made up of representatives from various county agencies and other support agencies. MEMA’s purpose is to ensure that the County of Maui is prepared to respond to any emergency or disaster that may occur.”


In [ ]:

user_question = "Summarize the officer accounts of the wildfire in Hawaii?"


eval_prompt = f"Just answer this question accurately: {user_question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Just answer this question accurately: Summarize the officer accounts of the wildfire in Hawaii?

Answer: On August 8, 2023, the officer accounts of the wildfire in Hawaii described a rapidly moving and dangerous fire that spread throughout the island of Maui, fueled by strong winds and dry conditions. The fire started in the afternoon and quickly grew in size, with flames reaching the outskirts of Lahaina town by the early evening. Officers reported heavy smoke, flames, and traffic congestion, making it difficult to evacuate residents. The officers also described a lack of cell service and internet connectivity, making it challenging to communicate with the public and other officers. Despite these challenges, the officers worked tirelessly to evacuate residents, respond to emergency calls, and manage traffic to help facilitate the evacuation of thousands of people. Through their efforts, many lives were saved, and the fire was eventually contained.


In [ ]:

user_question = "Tell me about the media coverage of the hawaii wildfires?"


eval_prompt = f"Just answer this question in less than 300 words: {user_question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Just answer this question in less than 300 words: Tell me about the media coverage of the hawaii wildfires?

Media coverage of the Hawaii wildfires was extensive, with both domestic and international news outlets covering the event. The fire was reported on by major US networks, such as ABC, CBS, NBC, and CNN, as well as international news outlets such as BBC, Al Jazeera, and Japanese television. Local news sources, such as Hawaii News Now, KITV, and Hawaii Public Radio, also provided extensive coverage of the fire. Social media was also used to disseminate information and updates, with the Hawaii Fire Department and other government agencies using Twitter and Facebook to post updates and fight rumors. Local newspapers, such as the Honolulu Star-Advertiser and the Hawaii Reporter, also covered the fire in their print editions. The fire was also covered by online news sources such as CNN, Fox News, and the New York Times. The media coverage of the fire helped keep the public informed an

In [ ]:

user_question = "Where were the media briefings held?"


eval_prompt = f"Just answer this question concisely: {user_question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Just answer this question concisely: Where were the media briefings held?

A. In the parking lot of the Lahaina Civic Center
B. In the Lahaina Civic Center itself
C. In the Lahaina Intermediate School parking lot
D. At the Lahaina Police Station

Answer: B. In the Lahaina Civic Center itself.


In [ ]:

user_question = "Tell me about Morgue Identification and Notification Task Force (M.I.N.T)?"


eval_prompt = f"Question: {user_question}. Just answer this question accurately and concisely\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Question: Tell me about Morgue Identification and Notification Task Force (M.I.N.T)?. Just answer this question accurately and concisely

Answer: The Morgue Identification and Notification Task Force (M.I.N.T.) was created by the County of Maui to identify and notify next of kin of decedents. M.I.N.T. operated from August 8, 2023 through approximately August 20, 2023. This team was created due to the overwhelming number of decedents that were recoveried and/or identified during the fires of August 8, 2023. The team was made up of two detectives, one forensic pathologist, one prosecuting attorney, one FBI agent, and one Clinical Labs of Hawaii representative. Their task was to review all recoveries and try to match decedents through dental charts, medical devices, personal effects, etc. The M.I.N.T. team also notified next of kin of decedents as their information was obtained. M.I.N.T. operated out of the Kaanapali Hotel. The team's efforts resulted in the identification of 90 decedents

In [ ]:

user_question = "When did the Hawaii wildfires take place?"


eval_prompt = f"Question: {user_question}. Just answer this question accurately and concisely\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Question: When did the Hawaii wildfires take place?. Just answer this question accurately and concisely

Answer: The Hawaii wildfires took place on August 8, 2023.


In [1]:

user_question = "Which geographic areas were affected by the wildfires in Hawaii in 2023? Give me some insights on this please?"


eval_prompt = f"Just answer this question concisely and accurately: {user_question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

NameError: name 'tokenizer' is not defined

In [ ]:
import os
print("=== Checking your finetunedModel directory ===")
print(os.listdir('./finetunedModel'))

print("\n=== Checking if logs directory exists ===")
if os.path.exists('./logs'):
    print(os.listdir('./logs'))
else:
    print("No logs directory found")

print("\n=== Checking sizes ===")
!du -sh ./finetunedModel/

=== Checking your finetunedModel directory ===
['checkpoint-485', 'checkpoint-1455', 'checkpoint-970']

=== Checking if logs directory exists ===
['events.out.tfevents.1757848046.7546e3df94dc.3639.0']

=== Checking sizes ===
349M	./finetunedModel/


In [ ]:
print("=== Detailed checkpoint information ===")
for checkpoint in ['checkpoint-485', 'checkpoint-970', 'checkpoint-1455']:
    checkpoint_path = f'./finetunedModel/{checkpoint}'
    print(f"\n{checkpoint}:")
    print(f"  Files: {os.listdir(checkpoint_path)}")

    # Check if trainer_state.json exists to see training info
    trainer_state_path = f'{checkpoint_path}/trainer_state.json'
    if os.path.exists(trainer_state_path):
        import json
        with open(trainer_state_path, 'r') as f:
            state = json.load(f)
        print(f"  Training step: {state.get('global_step', 'N/A')}")
        print(f"  Epoch: {state.get('epoch', 'N/A')}")
        if 'log_history' in state and state['log_history']:
            last_loss = state['log_history'][-1].get('train_loss', 'N/A')
            print(f"  Last training loss: {last_loss}")

print(f"\n=== Current working directory size ===")
!du -sh ./finetunedModel/checkpoint-*

=== Detailed checkpoint information ===

checkpoint-485:
  Files: ['optimizer.pt', 'training_args.bin', 'README.md', 'adapter_config.json', 'adapter_model.safetensors', 'special_tokens_map.json', 'scheduler.pt', 'tokenizer.model', 'rng_state.pth', 'tokenizer_config.json', 'chat_template.jinja', 'trainer_state.json']
  Training step: 485
  Epoch: 1.0
  Last training loss: N/A

checkpoint-970:
  Files: ['optimizer.pt', 'training_args.bin', 'README.md', 'adapter_config.json', 'adapter_model.safetensors', 'special_tokens_map.json', 'scheduler.pt', 'tokenizer.model', 'rng_state.pth', 'tokenizer_config.json', 'chat_template.jinja', 'trainer_state.json']
  Training step: 970
  Epoch: 2.0
  Last training loss: N/A

checkpoint-1455:
  Files: ['optimizer.pt', 'training_args.bin', 'README.md', 'adapter_config.json', 'adapter_model.safetensors', 'special_tokens_map.json', 'scheduler.pt', 'tokenizer.model', 'rng_state.pth', 'tokenizer_config.json', 'chat_template.jinja', 'trainer_state.json']
  Tra

In [ ]:
print("=== Creating complete backup ===")

# Create a backup with everything important
!zip -r my_hawaii_wildfire_model_backup.zip ./finetunedModel/ ./logs/

print("\n=== Checking backup size ===")
!ls -lh my_hawaii_wildfire_model_backup.zip

print("\n=== Backup created successfully! ===")
print("This contains:")
print("- All 3 checkpoints (485, 970, 1455)")
print("- Training logs")
print("- Tokenizer files")
print("- All adapter weights")

=== Creating complete backup ===
  adding: finetunedModel/ (stored 0%)
  adding: finetunedModel/checkpoint-485/ (stored 0%)
  adding: finetunedModel/checkpoint-485/optimizer.pt (deflated 10%)
  adding: finetunedModel/checkpoint-485/training_args.bin (deflated 54%)
  adding: finetunedModel/checkpoint-485/README.md (deflated 65%)
  adding: finetunedModel/checkpoint-485/adapter_config.json (deflated 57%)
  adding: finetunedModel/checkpoint-485/adapter_model.safetensors (deflated 7%)
  adding: finetunedModel/checkpoint-485/special_tokens_map.json (deflated 79%)
  adding: finetunedModel/checkpoint-485/scheduler.pt (deflated 61%)
  adding: finetunedModel/checkpoint-485/tokenizer.model (deflated 55%)
  adding: finetunedModel/checkpoint-485/rng_state.pth (deflated 26%)
  adding: finetunedModel/checkpoint-485/tokenizer_config.json (deflated 69%)
  adding: finetunedModel/checkpoint-485/chat_template.jinja (deflated 61%)
  adding: finetunedModel/checkpoint-485/trainer_state.json (deflated 78%)
  

In [ ]:
from google.colab import files
files.download('my_hawaii_wildfire_model_backup.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("=== Mounting Google Drive ===")
from google.colab import drive
drive.mount('/content/drive')

print("\n=== Creating directory in Drive ===")
!mkdir -p "/content/drive/MyDrive/FineTuned_Models/"

print("\n=== Copying model to Google Drive ===")
!cp -r ./finetunedModel "/content/drive/MyDrive/FineTuned_Models/hawaii_wildfire_llama2"
!cp -r ./logs "/content/drive/MyDrive/FineTuned_Models/hawaii_wildfire_llama2_logs"

print("\n=== Also copying the zip backup ===")
!cp my_hawaii_wildfire_model_backup.zip "/content/drive/MyDrive/FineTuned_Models/"

print("\n=== Verifying files saved to Drive ===")
!ls -la "/content/drive/MyDrive/FineTuned_Models/"

=== Mounting Google Drive ===
Mounted at /content/drive

=== Creating directory in Drive ===

=== Copying model to Google Drive ===

=== Also copying the zip backup ===

=== Verifying files saved to Drive ===
total 326264
drwx------ 5 root root      4096 Sep 14 13:27 hawaii_wildfire_llama2
drwx------ 2 root root      4096 Sep 14 13:27 hawaii_wildfire_llama2_logs
-rw------- 1 root root 334085219 Sep 14 13:27 my_hawaii_wildfire_model_backup.zip


In [ ]:
print("=== Installing Hugging Face Hub ===")
!pip install huggingface_hub

print("\n=== Login to Hugging Face ===")
from huggingface_hub import notebook_login
notebook_login()  # This will show a login widget

=== Installing Hugging Face Hub ===

=== Login to Hugging Face ===


In [ ]:
print("=== Getting your Hugging Face username ===")
from huggingface_hub import whoami

# Get your username
user_info = whoami()
username = user_info['name']
print(f"Your username: {username}")

print("\n=== Creating repository with correct name ===")
repo_name = f"{username}/hawaii-wildfire-llama2-qa"
print(f"Repository name: {repo_name}")

try:
    # Upload the fine-tuned adapter
    modelFinetuned.push_to_hub(repo_name, private=False)
    print("✅ Model uploaded successfully!")

    # Upload the tokenizer
    tokenizer.push_to_hub(repo_name, private=False)
    print("✅ Tokenizer uploaded successfully!")

    print(f"\n🎉 Your model is now available at: https://huggingface.co/{repo_name}")

except Exception as e:
    print(f"Error during upload: {e}")
    print("We might need to create the repository first...")

=== Getting your Hugging Face username ===
Your username: akshaygarur

=== Creating repository with correct name ===
Repository name: akshaygarur/hawaii-wildfire-llama2-qa


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...p9e9svs4h/adapter_model.safetensors:   0%|          | 49.2kB / 80.0MB            

✅ Model uploaded successfully!


...

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpzw361_6a/tokenizer.model      : 100%|##########|  500kB /  500kB            

  /tmp/tmpzw361_6a/tokenizer.model      : 100%|##########|  500kB /  500kB            

✅ Tokenizer uploaded successfully!

🎉 Your model is now available at: https://huggingface.co/akshaygarur/hawaii-wildfire-llama2-qa


In [ ]:
import torch

# Define the input prompt
prompt = "What causes wildfires in Hawaii?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output using the model
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=150)

# Decode and print the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Prompt:", prompt)
print("Response:", generated_text)


NameError: name 'tokenizer' is not defined